#### Instalar librerías

In [ ]:
!pip install openai
!pip install azure-ai-formrecognizer
!pip install azure-ai-textanalytics
!pip install azure-ai-vision-imageanalysis
!pip install faiss-cpu



  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/77/85/e7adeee84edd24c6cc119b2ccaaacd9579c6a2c7f72d05e936ea6b33594e/openai-1.54.3-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/65/9b/70f3ecbd3f18ef19e50fbe5a51bdb1c520282720896c16ae1a68b90675b8/jiter-0.7.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for typing-extensions<5,>=4.11 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/389.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/327.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 32.3 MB/s eta 0:00:00
  Attempting uninstall

#### Extraer información de los pdfs

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import os
import json

# Store connection information
endpoint = "https://westeurope.api.cognitive.microsoft.com/"
key = "API_KEY_AZURE"
path = "/Volumes/gestamp02/raw/gestamp02/financialReports/"  # (Primero tendremos que subir el PDF al Catalog)
output_path = "/Volumes/gestamp02/raw/gestamp02/financialReports/json/"

# Create the client
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Analyse the files
for pdf_file in os.listdir(path):
    print(pdf_file)
    if pdf_file.endswith(".pdf"):
        file_path = os.path.join(path, pdf_file)

        with open(file_path, "rb") as f:
          poller = document_analysis_client.begin_analyze_document(
                                          "prebuilt-layout",
                                          document=f
                                        )

        # Extract results from the document analysis
        layout = poller.result()

        # Convertir el resultado a un diccionario
        layout_dict = layout.to_dict()

        # Definir el nombre del archivo JSON de salida
        json_file_name = pdf_file.replace(".pdf", ".json")
        json_file_path = os.path.join(output_path, json_file_name)

        # Guardar el JSON en el directorio de salida
        with open(json_file_path, 'w') as fp:
            json.dump(layout_dict, fp)
        print(f"Guardado en {json_file_path}")

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:460)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:577)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:527)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:631)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:651)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

#### Sacar json de tablas

In [ ]:
# Load JSON file and convert it to a dictionary
import json
import os
import openai
import faiss
import numpy as np

Carpeta = ["Gestamp", "Magna", "Mercedes", "TataSteel"]
Documento = ["gestamp-fr.json", "magma-fr.json", "mercedes-benz-fr.json", "tata-steel-fr.json"]

for i in range(0, len(Carpeta)):
    output_path = "/Volumes/gestamp02/raw/gestamp02/financialReports/json/" + Carpeta[i] + "/"

    data_dict = {}
    with open("/Volumes/gestamp02/raw/gestamp02/financialReports/json/"+ Documento[i], 'r') as file:
        data_dict = json.load(file)
    print(data_dict.keys())

    for res in data_dict:
        if "tables" in res:
            #print(data_dict["tables"])
            for t in range(len(data_dict["tables"])):
                table = data_dict["tables"][t]["cells"]
                print("------------------------------------------------------")
                print(table)

                # Guardar el JSON en el directorio de salida
                json_file_path = os.path.join(output_path,  Carpeta[i] + "_" + str(t) + ".json")
                with open(json_file_path, 'w') as fp:
                    json.dump(table, fp)
                print(f"Guardado en {json_file_path}")


dict_keys(['api_version', 'model_id', 'content', 'languages', 'pages', 'paragraphs', 'tables', 'key_value_pairs', 'styles', 'documents'])
------------------------------------------------------
[{'kind': 'content', 'row_index': 0, 'column_index': 0, 'row_span': 1, 'column_span': 1, 'content': 'Preface', 'bounding_regions': [{'page_number': 2, 'polygon': [{'x': 10.7663, 'y': 1.0255}, {'x': 14.2426, 'y': 1.0159}, {'x': 14.2426, 'y': 1.4075}, {'x': 10.7663, 'y': 1.417}]}], 'spans': [{'offset': 77, 'length': 7}]}, {'kind': 'content', 'row_index': 0, 'column_index': 1, 'row_span': 1, 'column_span': 1, 'content': '4', 'bounding_regions': [{'page_number': 2, 'polygon': [{'x': 14.2426, 'y': 1.0159}, {'x': 14.5769, 'y': 1.0159}, {'x': 14.5769, 'y': 1.4075}, {'x': 14.2426, 'y': 1.4075}]}], 'spans': [{'offset': 85, 'length': 1}]}, {'kind': 'content', 'row_index': 1, 'column_index': 0, 'row_span': 1, 'column_span': 1, 'content': 'TATA STEEL NEDERLAND,', 'bounding_regions': [{'page_number': 2, 'poly

#### Transformar información

In [ ]:
import json
import os
import openai
import faiss
import numpy as np
import openai
import json
import pandas as pd
from openai import OpenAI
from openai import AzureOpenAI

array_kpis = []


# Setup client with Azure configurations
openai.api_type = "azure"
openai.api_deployment = "https://openaicore-sweden.openai.azure.com/"
openai.api_version = "2024-02-15-preview"
openai.api_key = "API_KEY_OPENAI"


# Definimos la función para obtener los embeddings
def get_embedding(text, model="text-embedding-3-large"):
  embeddings = client.embeddings.create(
      model=model,
      input=text
  )
  return embeddings.data[0].embedding

# Crear una lista para guardar los embeddings y sus textos
all_embeddings = []
all_texts = []

# Creamos una instancia de cliente de AzureOpenAI
client = AzureOpenAI(
  azure_endpoint = "https://openaicore-sweden.openai.azure.com/", # El endpoint a usar lo daremos al inicio de la competición
  api_key="API_KEY_AZURE",  # La clave también os la daremos al comenzar la competición
  api_version="2024-02-15-preview"
)

# Carpetas y documentos
Carpeta = ["Gestamp", "Magna", "Mercedes", "TataSteel"]
Documento = ["gestamp-fr.json", "magma-fr.json", "mercedes-benz-fr.json", "tata-steel-fr.json"]

for i in range(0, len(Carpeta)):
  path = "/Volumes/gestamp02/raw/gestamp02/financialReports/json/" + Carpeta[i] + "/"
  j = 0

  for archivo in os.listdir(path):
    file_path = os.path.join(path, archivo)

    with open(file_path, 'r') as file:
        data_dict = json.load(file)

    tabla_json = data_dict


    # Suponiendo que el JSON está almacenado en una variable llamada `data`
    data = tabla_json

    # Crear una estructura de diccionario para organizar el contenido por filas y columnas
    table = {}

    # Recorremos cada entrada en el JSON
    for entry in data:
        row_index = entry["row_index"]
        col_index = entry["column_index"]
        content = entry["content"]

        # Asegurarse de que la fila existe en el diccionario
        if row_index not in table:
            table[row_index] = {}

        # Agregar el contenido en la posición adecuada de la fila y columna
        table[row_index][col_index] = content

    # Convertimos el diccionario a un DataFrame de pandas
    df = pd.DataFrame.from_dict(table, orient="index").sort_index(axis=0).sort_index(axis=1)
    #print(df)
    all_table = ""
    for index, row in df.iterrows():
        # Unir todas las celdas de la fila en una cadena de texto
        text_representation = " ".join([str(cell) for cell in row if pd.notna(cell)])
        all_table+=text_representation
        all_table+="\n"

    print(all_table)
    formato_json = """{
        "kpi" : "",
        "Empresa : """+ Carpeta[i] + """,
        "valores" : {
            "2023" : valor,
            "2022" : valor,
            "2021" : valor
        }
    }"""

    prompt = f"A partir de la tabla definida en este archivo que recoge los datos de una tabla indicando diferentes kpis: {all_table}. Genera un archivo JSON de salida, que tenga el siguiente formato: {formato_json}. Engloba todos los objetos json devuelve unicamente dicho archivo json."

    #print(prompt)
    messages = []
    messages.append({"role": "user", "content": prompt})

    # Para obtener una respuesta nueva del modelo, enviamos la lista de mensajes. También podemos configurar diversos parámetros para afinar
    # el comportamiento del modelo

    completion = client.chat.completions.create(
      model="gpt-4-turbo",
      messages = messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
    )

    #print("Respuesta del chat=")
    response = completion.choices[0].message.content
    print(response)

    outgoing_path = path + "/txt/" + str(j) + ".txt"
    with open(outgoing_path, 'w') as fp:
            fp.write(response)
    j+=1
  #embedding_res = get_embedding(response)

Preface 4
TATA STEEL NEDERLAND, 6
PRIORITIES AND GOVERNANCE
1
1.1 Tata Steel Nederland 8
1.2 Our objective, our mission, our vision 12
1.3 Sustainability strategy 14
1.4 Stakeholder dialogue 16
1.5 Risk management 18
1.6 Governance 20
CUSTOMER & VALUE
2 22
2.1 The added value of steel 24
2.2 Sustainability in our steel 28
2.3 Investing in the future 32
ENVIRONMENT & COMMUNITY
3 34
3.1 Roadmap Plus 36
3.2 Monitoring and measuring 46
3.3 Investigating our impact 48
3.4 Sustainable relationship with the community 50
DECARBONISATION & SUSTAINABILITY
4 54
4.1 Current carbon footprint 56
4.2 Reorientation of the climate strategy 60
4.3 Climate strategy 62
4.4 Energy efficiency 68
4.5 Raw materials efficiency 70
4.6 Responsible sourcing 72
PEOPLE & SOCIETY
5 78
5.1 Employees make the company 80
5.2 Health and Safety at work 82
5.3 Health and vitality 86
5.4 Equal opportunities 88
5.5 Training 90
HOW WE REPORT
6 94
- Key figures 96
- GRI Content Index Tata Steel - Core 102
- Advisory bodies, i

---------------------------------------------------------------------------
IsADirectoryError                         Traceback (most recent call last)
File <command-4399001801260833>, line 54
     51 for archivo in os.listdir(path):
     52   file_path = os.path.join(path, archivo)
---> 54   with open(file_path, 'r') as file:
     55       data_dict = json.load(file)
     57   tabla_json = data_dict

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:286, in _modified_open(file, *args, **kwargs)
    279 if file in {0, 1, 2}:
    280     raise ValueError(
    281         f"IPython won't let you open fd={file} by default "
    282         "as it is likely to crash IPython. If you know what you are doing, "
    283         "you can use builtins' open."
    284     )
--> 286 return io_open(file, *args, **kwargs)

IsADirectoryError: [Errno 21] Is a directory: '/Volumes/gestamp02/raw/gestamp02/financialReports/json/TataSteel/txt'

#### Resumir información

In [ ]:
# Setup client with Azure configurations
openai.api_type = "azure"
openai.api_deployment = "https://openaicore-sweden.openai.azure.com/"
openai.api_version = "2024-02-15-preview"
openai.api_key = "API_KEY_OPENAI"

# Creamos una instancia de cliente de AzureOpenAI
client = AzureOpenAI(
  azure_endpoint = "https://openaicore-sweden.openai.azure.com/", # El endpoint a usar lo daremos al inicio de la competición
  api_key="API_KEY_AZURE",  # La clave también os la daremos al comenzar la competición
  api_version="2024-02-15-preview"
)

outgoing_path = "/Volumes/gestamp02/raw/gestamp02/financialReports/KPIs_principales/"

# Carpetas y documentos
Carpeta = ["Gestamp", "Magna", "Mercedes", "TataSteel"]
Documento = ["gestamp-fr.json", "magma-fr.json", "mercedes-benz-fr.json", "tata-steel-fr.json"]
contenido_archivos = []
for i in range(0, len(Carpeta)):
  path = "/Volumes/gestamp02/raw/gestamp02/financialReports/json/" + Carpeta[i] + "/" + "txt/"

  # Recorre todos los archivos en la carpeta especificada
  for nombre_archivo in os.listdir(path):
      # Verifica que el archivo sea un .txt
      if nombre_archivo.endswith(".txt"):
          ruta_archivo = os.path.join(path, nombre_archivo)

          # Lee el contenido del archivo y agrégalo a la lista
          with open(ruta_archivo, "r") as archivo:
              contenido = archivo.read()
              contenido_archivos.append(contenido)

  prompt = f"A partir del array definido en esta variable {contenido_archivos}, quiero que obtengas la información más relevante, clasificada por KPIs. No pueden faltar información sobre emisiones, Agua y numero de trabajadoras femeninas y numero de trabajadores totales de la empresa (Alomejor estas variables se encuentran en inglés pero tradúcelas al castellano). Como respuesta quiero un json que almacene esto."

  #print(prompt)
  messages = []
  messages.append({"role": "user", "content": prompt})

  # Para obtener una respuesta nueva del modelo, enviamos la lista de mensajes. También podemos configurar diversos parámetros para afinar
  # el comportamiento del modelo

  completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages = messages,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
  )

  #print("Respuesta del chat=")
  response = completion.choices[0].message.content
  #print(response)

  json_file_path = os.path.join(outgoing_path, Carpeta[i] + ".json")
  with open(json_file_path, 'w') as fp:
    json.dump(response, fp)




####Combinación

In [ ]:
# Setup client with Azure configurations
openai.api_type = "azure"
openai.api_deployment = "https://openaicore-sweden.openai.azure.com/"
openai.api_version = "2024-02-15-preview"
openai.api_key = "API_KEY_OPENAI"
contenido_archivos = []
# Creamos una instancia de cliente de AzureOpenAI
client = AzureOpenAI(
  azure_endpoint = "https://openaicore-sweden.openai.azure.com/", # El endpoint a usar lo daremos al inicio de la competición
  api_key="API_KEY_AZURE",  # La clave también os la daremos al comenzar la competición
  api_version="2024-02-15-preview"
)

path = "/Volumes/gestamp02/raw/gestamp02/financialReports/KPIs_principales/"

# Recorre todos los archivos en la carpeta especificada
for nombre_archivo in os.listdir(path):
    # Verifica que el archivo sea un .txt
    if nombre_archivo.endswith(".json"):
        ruta_archivo = os.path.join(path, nombre_archivo)

        # Lee el contenido del archivo y agrégalo a la lista
        with open(ruta_archivo, "r") as archivo:
            contenido = json.load(archivo)
            contenido_archivos.append(contenido)

prompt = f"A partir del array definido en esta variable {contenido_archivos}, quiero que hagas una tabla para unir toda la informacion. Quiero que haya una columna para cada empresa y que en filas se intenten unificar los datos. Si los datos no están completos, pon un 0 en las columnas vacías pero información en los que sí haya información. Como respuesta quiero un json que almacene todo esto"

messages = []
messages.append({"role": "user", "content": prompt})

# Para obtener una respuesta nueva del modelo, enviamos la lista de mensajes. También podemos configurar diversos parámetros para afinar
# el comportamiento del modelo

completion = client.chat.completions.create(
  model="gpt-4-turbo",
  messages = messages,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

#print("Respuesta del chat=")
response = completion.choices[0].message.content
#print(response)

json_file_path = os.path.join(path + "InformacionEmpresas.json")
with open(json_file_path, 'w') as fp:
  json.dump(response, fp)



Preguntas

In [ ]:
# Setup client with Azure configurations
openai.api_type = "azure"
openai.api_deployment = "https://openaicore-sweden.openai.azure.com/"
openai.api_version = "2024-02-15-preview"
openai.api_key = "API_KEY_OPENAI"

# Creamos una instancia de cliente de AzureOpenAI
client = AzureOpenAI(
  azure_endpoint = "https://openaicore-sweden.openai.azure.com/", # El endpoint a usar lo daremos al inicio de la competición
  api_key="API_KEY_AZURE",  # La clave también os la daremos al comenzar la competición
  api_version="2024-02-15-preview"
)

archivo = "/Volumes/gestamp02/raw/gestamp02/financialReports/KPIs_principales/InformacionEmpresas.json"
archivo_salida = "/Volumes/gestamp02/raw/gestamp02/financialReports/KPIs_principales/Preguntas/"

with open(archivo, 'r') as file:
    contenido = json.load(file)

prompt1 = f"A partir del array definido en esta variable {contenido}, quiero que hagas una estimacion de los valores que va a tener en los siguientes dos años. A la información que te paso por {contenido}, añade unas nuevas columnas "


messages1 = []
messages1.append({"role": "user", "content": prompt1})


# Para obtener una respuesta nueva del modelo, enviamos la lista de mensajes. También podemos configurar diversos parámetros para afinar
# el comportamiento del modelo
completion1 = client.chat.completions.create(
  model="gpt-4-turbo",
  messages = messages1,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

#print("Respuesta del chat=")
response1 = completion1.choices[0].message.content

json_file_path = os.path.join(archivo_salida + "Pregunta1.json")
with open(json_file_path, 'w') as fp:
  json.dump(response1, fp)

